In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot = True)

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28*28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32,[None,n_input])
Y = tf.placeholder(tf.float32,[None,n_class])
Z = tf.placeholder(tf.float32,[None,n_noise])

C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise,labels],1)
        hidden = tf.layers.dense(inputs,n_hidden,activation = tf.nn.relu)
        output = tf.layers.dense(hidden,n_input,activation = tf.nn.sigmoid)
    return output
def discriminator(inputs,labels,reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs,labels],1)
        hidden = tf.layers.dense(inputs,n_hidden,activation = tf.nn.relu)
        output = tf.layers.dense(hidden,1,activation = None)
    return output
def get_noise(batch_size,n_noise):
    return np.random.uniform(-1.,1.,size=[batch_size,n_noise])

In [3]:
G = generator(Z,Y)
D_real = discriminator(X,Y)
D_gene = discriminator(G,Y,True)
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.ones_like(D_gene)))

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='generator')
train_D = tf.train.AdamOptimizer().minimize(loss_D,var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,var_list=vars_G)

In [5]:
sess =tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D,loss_val_G = 0,0
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size,n_noise)
        _,loss_val_D = sess.run([train_D,loss_D],feed_dict={X:batch_xs,Y:batch_ys,Z:noise})
        _,loss_val_G = sess.run([train_G,loss_G],feed_dict={X:batch_xs,Y:batch_ys,Z:noise})
    print('epoch:','%04d'%epoch,'D_loss: {:4f}'.format(loss_val_D),'G_loss: {:4f}'.format(loss_val_G))
    if epoch ==0 or(epoch+1) %10 == 0:
        sample_size = 10
        noise = get_noise(sample_size,n_noise)
        samples = sess.run(G,feed_dict={Y:mnist.test.labels[:sample_size],Z:noise})
        fig,ax = plt.subplots(2,sample_size,figsize =(sample_size,2))
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            ax[0][i].imshow(np.reshape(mnist.test.images[i],(28,28)))
            ax[1][i].imshow(np.reshape(samples[i],(28,28)))
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
        plt.close(fig)

epoch: 0000 D_loss: 0.010772 G_loss: 7.362059
epoch: 0001 D_loss: 0.011307 G_loss: 8.066823
epoch: 0002 D_loss: 0.005449 G_loss: 7.483088
epoch: 0003 D_loss: 0.002767 G_loss: 8.613208
epoch: 0004 D_loss: 0.001598 G_loss: 8.681866
epoch: 0005 D_loss: 0.003167 G_loss: 9.713061
epoch: 0006 D_loss: 0.007072 G_loss: 10.246282
epoch: 0007 D_loss: 0.027725 G_loss: 7.643903
epoch: 0008 D_loss: 0.013519 G_loss: 7.451093
epoch: 0009 D_loss: 0.098468 G_loss: 6.815314
epoch: 0010 D_loss: 0.139810 G_loss: 6.417241
epoch: 0011 D_loss: 0.177849 G_loss: 6.009683
epoch: 0012 D_loss: 0.252927 G_loss: 4.685071
epoch: 0013 D_loss: 0.298149 G_loss: 4.567585
epoch: 0014 D_loss: 0.305299 G_loss: 4.592999
epoch: 0015 D_loss: 0.335279 G_loss: 3.717649
epoch: 0016 D_loss: 0.336008 G_loss: 4.314379
epoch: 0017 D_loss: 0.382780 G_loss: 3.434916
epoch: 0018 D_loss: 0.526414 G_loss: 4.015527
epoch: 0019 D_loss: 0.443158 G_loss: 3.848062
epoch: 0020 D_loss: 0.654906 G_loss: 3.038048
epoch: 0021 D_loss: 0.669371 G_lo